# Proof of burn

In [ ]:
%pip install ecdsa

In [ ]:
import sys
from pathlib import Path

sys.path.append(str(Path().resolve().parent.parent))

from elliptic_curves.instantiations.mnt4_753.mnt4_753 import MNT4_753
from tx_engine import Script, Tx, TxOut, Wallet

from token_examples.proof_of_burn.utils import generate_pob_utxo, load_and_process_vk, spend_proof_of_burn
from token_examples.tx_engine_utils import (
    p2pkh,
    setup_network_connection,
    spend_p2pkh,
    tx_from_id,
    tx_to_input,
    update_tx_balance,
)

ScalarFieldMNT4 = MNT4_753.scalar_field

In [3]:
# Connection to the blockchain
connection = setup_network_connection("testnet")

In [ ]:
# The public key of the token holder
token_pub_key = Wallet.from_hexstr("BSV_Testnet", "")
token_pub_key.get_address()

'mhtNYK9vjPZuKiXibmUpRzwebJi4sW89GS'

In [ ]:
# The public key of the token holder
issuer_pub_key = Wallet.from_hexstr("BSV_Testnet", "")
issuer_pub_key.get_address()

'mySeWDWTQkZRroRoEghhq8FEuvBV8p5pLb'

In [ ]:
genesis_tx = tx_from_id("", connection)
issuer_funds = tx_from_id("", connection)

## Generate proof-of-burn UTXO

In [37]:
genesis_txid_as_input = ScalarFieldMNT4(int.from_bytes(genesis_tx.hash(), "little"))
vk, cache_vk, prepared_vk = load_and_process_vk(genesis_txid=genesis_txid_as_input.to_int())

In [38]:
proof_of_burn_utxo, change_utxo = generate_pob_utxo(
    vk=vk, prepared_vk=prepared_vk, issuer_funds=issuer_funds, issuer_pub_key=issuer_pub_key
)

In [39]:
proof_of_burn_tx, response = spend_p2pkh(
    txs=[issuer_funds],
    indices=[0],
    outputs=[proof_of_burn_utxo, change_utxo],
    index_output=1,
    public_keys=[issuer_pub_key],
    fee_rate=10,
    network=connection,
)

assert response.status_code == 200, f"Failed to create proof of burn tx, error is {response.content}"

In [40]:
proof_of_burn_tx.id()

'3718ad6c4abda6fa56490dce528dbcae8f8b467ec07f2b07c41481e225f53e37'

## Move the token two times

In [41]:
token_tx_1, response = spend_p2pkh(
    txs=[genesis_tx],
    indices=[0],
    outputs=[p2pkh(token_pub_key, genesis_tx.tx_outs[0].amount)],
    index_output=0,
    public_keys=[token_pub_key],
    fee_rate=10,
    network=connection,
)

assert response.status_code == 200, f"Token transfer failed, error is: {response.content}"

In [42]:
token_tx_1.serialize().hex()

'0100000001a4eeb83545a18d17da630c6722b6f1150bf43ded1a1df1a8466887c2daa1cb10000000006b483045022100e181d2380d21ad92a0f88bc8dd47f49998fd02ffc82dcf69fc34fcb1a54f378c0220277d77e64d83a7463b595d9c2cd0fbd22ede4aac131a512674aebfc4d9286d4741210255f37269612cfafd64256a16dbd8f6fb1cb550f1e415c9c0d24ba9021f7ff18000000000013e860100000000001976a91419fd39e77e7c28969a5f37b180dfad76c807097b88ac00000000'

In [43]:
token_tx_1.id()

'3e8150d2bc7aedc9f360f40f5b94821dde376658c82f0831c8e78fbee5d3b457'

In [44]:
token_tx_2, response = spend_p2pkh(
    txs=[token_tx_1],
    indices=[0],
    outputs=[p2pkh(token_pub_key, token_tx_1.tx_outs[0].amount)],
    index_output=0,
    public_keys=[token_pub_key],
    fee_rate=10,
    network=connection,
)

assert response.status_code == 200, f"Token transfer failed, error is: {response.content}"

In [45]:
token_tx_2.serialize().hex()

'010000000157b4d3e5be8fe7c831082fc8586637de1d82945b0ff460f3c9ed7abcd250813e000000006b48304502210093b900291c7c588544029118bb95dd5a4dd3c365c2cb3b3ca1c17ff9b145e4e7022027f802f68eaf43f30d530f79ad2235bde4dd6d36840feba284a836ce4bf5559e41210255f37269612cfafd64256a16dbd8f6fb1cb550f1e415c9c0d24ba9021f7ff18000000000013c860100000000001976a91419fd39e77e7c28969a5f37b180dfad76c807097b88ac00000000'

In [46]:
token_tx_2.id()

'030d5226a05135c51b702a2043064cd0b37c1b6422d871e3dc69e668d4ed0e70'

## Burning transaction

In [47]:
inputs = [
    tx_to_input(token_tx_2, 0, Script()),
    tx_to_input(proof_of_burn_tx, 0, Script()),
    tx_to_input(proof_of_burn_tx, 1, Script()),
]

outputs = [
    TxOut(amount=0, script_pubkey=Script.parse_string("OP_0 OP_RETURN")),
    p2pkh(issuer_pub_key, amount=proof_of_burn_tx.tx_outs[1].amount + token_tx_2.tx_outs[0].amount),
]

spending_tx = Tx(
    version=1,
    tx_ins=inputs,
    tx_outs=outputs,
    locktime=0,
)

# Fee rate chosen to get 10 sat/KB after adding the unlocking script
spending_tx = update_tx_balance(spending_tx, 1, (178 + 300000) // 178 * 10)

In [48]:
spending_tx.serialize().hex()

'0100000003700eedd468e669dce371d822641b7cb3d04c0643202a701bc53551a026520d03000000000000000000373ef525e28114c4072b7fc07e468b8faebc8d52ce0d4956faa6bd4a6cad1837000000000000000000373ef525e28114c4072b7fc07e468b8faebc8d52ce0d4956faa6bd4a6cad183701000000000000000002000000000000000002006aecef0200000000001976a914c4a1bfe9fc4e17cffea66f92820068b45a546b8988ac00000000'

In [49]:
# Spend Proof-of-Burn UTXO
spending_tx, response = spend_proof_of_burn(
    outputs=spending_tx.tx_outs,
    cache_vk=cache_vk,
    token_tx=token_tx_2,
    pob_tx=proof_of_burn_tx,
    token_pub_key=token_pub_key,
    pob_pub_key=issuer_pub_key,
    network=connection,
)

assert response.status_code == 200, f"Spending of PoB failed, error is: {response.content}"

In [50]:
spending_tx.id()

'5fff4a7a953b5819ac2ab4b35dd9d9b9a18d18e40a3d87917fbb0a7a930b90ea'